In [21]:
import os, errno
import re
import shutil
import tarfile
import nglview as nv
from Bio import pairwise2
from Bio import SeqIO
import MDAnalysis as mda
from MDAnalysis.analysis import align
from modeller_script.evaluate_modeller import evaluate_modeller

In [ ]:
def create_directory(directory):
    try:
        os.makedirs(directory)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    
    return directory

In [ ]:
def move_files(source, dest, file_name):
    files = os.listdir(source)
    for f in files:
        if re.match(file_name, f):
            shutil.move(source+f, dest)

In [136]:
DOWNLOADS = '/Users/evabertalan/Downloads'

### 1. prepare directory

In [42]:
PDB_CODE = '6b73'
CHAIN = 'B'

In [43]:
folder_name = PDB_CODE+CHAIN
path = '../'+folder_name

In [ ]:
directory = create_directory(path)

### 2. download pdb file and fasta sequence:

In [ ]:
file_name = PDB_CODE+'.pdb'
! (cd $directory && curl -O https://files.rcsb.org/download/$file_name)

In [ ]:
fasta_path = directory+'/'+PDB_CODE+'.fasta'
fasta_url = '"'+'https://www.rcsb.org/pdb/download/downloadFile.do?fileFormat=fastachain&compression=NO&structureId='+PDB_CODE+'&chainId='+CHAIN+'"'        
! curl -o $fasta_path $fasta_url

### 3. create sequence:

In [ ]:
! (cd $directory && mod9.21 ../code/modeller_script/get_seq.py $PDB_CODE $CHAIN)

### 4. create alignement:

In [83]:
def create_alignment(pdb_code, chain, directory):
    sequence_path = directory+'/'+pdb_code+'.seq'
    ali_path = directory+'/alignment.ali'

    sequence = ''
    fasta = SeqIO.read(directory+'/'+PDB_CODE+'.fasta', 'fasta').seq

    with open(sequence_path) as sequence_file:
        alignment_file = open(ali_path, 'w+')
        for i, line in enumerate(sequence_file):
            if i < 3:
                alignment_file.write(line)
            else:
                sequence += line
                
        sequence = sequence[:-2]        
        alignments = pairwise2.align.globalxx(fasta, sequence)

        alignment_file.write(alignments[0][1])
        alignment_file.write('*\n')
        alignment_file.write('>P1;'+pdb_code+'_fill\n')
        alignment_file.write('sequence:::::::::\n')
        alignment_file.write(str(fasta)+'*')
        alignment_file.close()

In [84]:
create_alignment(PDB_CODE, CHAIN, directory)

### 5. compose template and target sequence:
	template: >P1;6b73
				structureX:6b73:
				- - - for missing residues
	target: >P1;6b73_fill
				sequence:::::::::
				insert residues where it missing;
				- - - for residues what are not needed

In [85]:
! open -e $ali_path
! /Applications/Chimera.app/Contents/MacOS/chimera $directory/$file_name

### 6. run modeller:
 * knowns = sys.argv[1]
 * sequence = sys.argv[2]
 * num_models = sys.argv[3]
 * num_loops = sys.argv[4]

In [ ]:
pdb_fill = PDB_CODE+'_fill'
! (cd $directory && mod9.21 ../code/modeller_script/loopmodel.py $PDB_CODE $pdb_fill 20 4)

### 7. evaluate modeller result:

In [5]:
file_name = pdb_fill+'.'
best_model, best_loop = evaluate_modeller(file_name, loop=True)

            DOPE  ga341       molpdf                     name  norm_DOPE
5  -38197.539062    1.0   919.880127  6b73_fill.B99990006.pdb   -0.55493
7  -38087.347656    1.0   918.481445  6b73_fill.B99990008.pdb   -0.52706
3  -38080.039062    1.0   906.115295  6b73_fill.B99990004.pdb   -0.52521
14 -38067.464844    1.0   955.168335  6b73_fill.B99990015.pdb   -0.52203
13 -37928.148438    1.0   962.934448  6b73_fill.B99990014.pdb   -0.48679
9  -37927.113281    1.0   984.575317  6b73_fill.B99990010.pdb   -0.48653
18 -37926.093750    1.0   978.404114  6b73_fill.B99990019.pdb   -0.48627
16 -37916.238281    1.0   973.556519  6b73_fill.B99990017.pdb   -0.48378
17 -37885.402344    1.0   884.425964  6b73_fill.B99990018.pdb   -0.47598
2  -37876.269531    1.0   979.899719  6b73_fill.B99990003.pdb   -0.47367
12 -37875.011719    1.0  1070.296997  6b73_fill.B99990013.pdb   -0.47335
4  -37871.675781    1.0  1028.565430  6b73_fill.B99990005.pdb   -0.47251
11 -37859.902344    1.0  1005.030090  6b73_fill.B99

In [6]:
! /Applications/Chimera.app/Contents/MacOS/chimera $directory/$best_model $directory/$best_loop

In [19]:
def copy_best(source, dest, file_name, best):
    files = os.listdir(source)
    for f in files:
        if re.match(best, f):
            shutil.copy(source+'/'+f, dest+file_name+'pdb')

In [20]:
model_folder = create_directory(directory+'/models')
move_files(directory+'/', model_folder, file_name, best_model)
copy_best(model_folder, directory+'/', file_name, best_model)
copy_best(model_folder, directory+'/', file_name[:-1]+'_loop.', best_loop)

In [151]:
! cp ./modeller_script/loopmodel.log $model_folder

### 8. upload to OPM:
PDB_CODE_fill.pdb
https://opm.phar.umich.edu/ppm_server

In [21]:
new_opm_name = directory+'/'+PDB_CODE+'_fill_opm.pdb'
! cp $DOWNLOADS/*_fill.pdb $new_opm_name

In [24]:
new_loop_opm_name = directory+'/'+PDB_CODE+'_fill_loop_opm.pdb'
! cp $DOWNLOADS/*_fill_loop.pdb $new_loop_opm_name

In [ ]:
! rm $DOWNLOADS/*_fill.pdb
! rm $DOWNLOADS/*_fill_loop.pdb

### 9. compare the oriented structure with the original opm:

In [116]:
original_opm = '../opm/'+PDB_CODE+'.pdb'

In [25]:
! /Applications/Chimera.app/Contents/MacOS/chimera $directory/$new_loop_opm_name $original_opm

### 10. remove HETATMs from pdb:

In [26]:
with open(new_loop_opm_name) as opm_file:
    input_file = open(directory+'/'+PDB_CODE+'_inp.pdb', 'w+')
    for i, line in enumerate(opm_file):
        if not re.match('HETATM', line):
            input_file.write(line)
    input_file.write('END')
    input_file.close()

In [87]:
inp_file = PDB_CODE+'_inp.pdb'
! open -e $directory/$inp_file

In [118]:
view = nv.show_file(directory+'/'+PDB_CODE+'_inp.pdb')
view

A Jupyter Widget

### 11. upload to charmm-gui:
http://www.charmm-gui.org/?doc=input/membrane

#### after step 3 check packing:

In [130]:
tar = tarfile.open(DOWNLOADS+'/charmm-gui.tgz', 'r:gz')
for member in tar.getmembers():
    if re.search('step3_packing.pdb', member.name):
        f = tar.extract(member, 'temp')
f = [i for i in os.listdir('temp') if re.match('charmm-gui', i)]
step3_pdb = 'temp/'+f[0]+'/step3_packing.pdb'

In [132]:
! /Applications/Chimera.app/Contents/MacOS/chimera $step3_pdb $original_opm

In [ ]:
! rm $DOWNLOADS/charmm-gui.tgz

### 12. prepare charmm-gui to NAMD 

In [135]:
! cp $DOWNLOADS/charmm-gui.tgz $directory

cp: /Users/evabertalan/Downloads/charmm-gui.tgz: No such file or directory


In [200]:
tar = tarfile.open(directory+'/charmm-gui.tgz', 'r:gz')
tar.extractall(directory)
charmm_folder = [i for i in os.listdir(directory) if re.match('charmm-gui-', i)][0]
namd_folder = directory+charmm_folder+'/namd/'
inp_files = sorted([namd_folder+i for i in os.listdir(namd_folder) if re.match(r'(step6.).*\_equilibration.inp$', i)])
prod_file = namd_folder+'step7.1_production.inp'

In [201]:
PMEGridSize = '120'
langevinDamping = '5.0'

In [207]:
def write_namd_input(inp_files):
    for inp_file in inp_files:
        with open(inp_file, 'r+') as f:
            content = f.readlines()
            output = ''
            for i, line in enumerate(content):
                if re.match('wrapWater', line):
                    output += 'wrapWater   off;\n'

                elif re.match('wrapAll', line):
                    output += 'wrapAll   off;\n'

                elif re.match('wrapNearest', line):
                    output += 'wrapNearest   off;\n'

                elif re.match('PMEGridSpacing', line):
                    output += 'PMEGridSizeX   '+PMEGridSize+';\n'
                    output += 'PMEGridSizeY   '+PMEGridSize+';\n'
                    output += 'PMEGridSizeZ   '+PMEGridSize+';\n'

                elif re.match('langevinDamping', line):
                    output += 'langevinDamping   '+langevinDamping+';\n'

                else:
                    output += line
            f.seek(0)
            f.write(output)

In [ ]:
write_namd_input(inp_files)

In [208]:
write_namd_input([prod_file])

### 13. create job.sh

In [224]:
def create_job_script(cluster, step):
    files = os.listdir('job_scripts')
    file_name = cluster+'_'+step+'.sh'
    for f in files:
        if re.match(file_name, f):
            shutil.copy('job_scripts/'+file_name, namd_folder+step+'_job.sh')

* cluster = 'hlrn', 'leonard'
* step = 'eq', 'prod'

In [223]:
 create_job_script('hlrn', 'eq')

### 14. upload charrm folder to cluster

* check input files
* set numsteps
* update job.sh
* scp to cluster
* run namd